In [2]:
import os
os.environ['MPLBACKEND'] = 'Agg'

In [3]:
!ls

all_score.py		    competition_support_set	    ruff.toml
assets			    competition_support_set.zip     run3090.ipynb
baseline		    configs			    scripts
baseline2		    debug_eval.py		    server_train_2.py
baseline2_eval		    debug_pred.h5ad		    server_train.py
baseline_eval		    diagnose_overlap.py		    singularity.def
batch_eval.py		    examples			    src
cell-eval-outdir	    LICENSE			    tests
check_competition_data.sh   MODEL_ACCEPTABLE_USE_POLICY.md  testtt.ipynb
check_h5_files.py	    MODEL_LICENSE.md		    train_and_eval.py
competition		    my_real_data.h5ad		    training.out
competition_40000	    --outdir			    uv.lock
competition_40000_500	    pyproject.toml		    wandb
competition_hepg2	    README.md			    wandb_logs
competition_hepg2_2	    recover_missing_logs.py
competition_hepg2_official  results


In [4]:
%pip install requests tqdm

import requests
from tqdm.auto import tqdm  # picks the best bar for the environment

Note: you may need to restart the kernel to use updated packages.


/media/mldadmin/home/s125mdg34_03/miniconda3/envs/state_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# install the package
! uv run state --help

usage: state [-h] {emb,tx} ...

positional arguments:
  {emb,tx}

options:
  -h, --help  show this help message and exit


In [1]:
# 添加 WANDB_MODE=disabled 环境变量来强制关闭 WandB
# 注意：保留 wandb.project 等参数以防止配置报错
! WANDB_MODE=disabled uv run state tx train \
  data.kwargs.toml_config_path="competition_support_set/server_config_5.toml" \
  data.kwargs.num_workers=8 \
  data.kwargs.batch_col="batch_var" \
  data.kwargs.pert_col="target_gene" \
  data.kwargs.cell_type_key="cell_type" \
  data.kwargs.control_pert="non-targeting" \
  data.kwargs.perturbation_features_file="competition_support_set/ESM2_pert_features.pt" \
  training.max_steps=40000 \
  training.ckpt_every_n_steps=1000 \
  model=state_sm \
  wandb.tags="[run_3090]" \
  wandb.project=vcc \
  wandb.entity=arcinstitute \
  output_dir="competition" \
  name="run_1230"

Seed set to 42
competition_support_set/{competition_train,rpe1,jurkat,k562,hepg2}.h5
Dataset path does not exist: competition_support_set/{competition_train,rpe1,jurkat,k562,hepg2}.h5
Processed competition_train: 221273 train, 0 val, 0 test                        
Processing replogle_h1:  20%|████▏                | 1/5 [00:00<00:00,  6.76it/s]No cell barcode information found in competition_support_set/rpe1.h5. Generating generic barcodes.
Processed rpe1: 22317 train, 0 val, 0 test                                      
Processing replogle_h1:  20%|████▏                | 1/5 [00:00<00:00,  6.76it/s]No cell barcode information found in competition_support_set/jurkat.h5. Generating generic barcodes.
Processed jurkat: 21412 train, 0 val, 0 test                                    
Processing replogle_h1:  20%|████▏                | 1/5 [00:00<00:00,  6.76it/s]No cell barcode information found in competition_support_set/k562.h5. Generating generic barcodes.
Processed k562: 18465 train, 0 val

In [1]:
# Run inference on validation anndata - make sure the checkpoint exists
! WANDB_MODE=offline uv run state tx infer \
  --output "competition/first_run/prediction_8000.h5ad" \
  --model-dir "competition/first_run" \
  --checkpoint "competition/first_run/checkpoints/step=8000.ckpt" \
  --adata "competition_support_set/competition_val_template.h5ad" \
  --pert-col "target_gene"

==> STATE: tx infer (virtual experiment)
Loaded config: competition/first_run/config.yaml
Control perturbation: non-targeting
Grouping by cell type column: cell_type
StateTransitionPerturbationModel(
  (loss_fn): SamplesLoss()
  (pert_encoder): Sequential(
    (0): Linear(in_features=5120, out_features=672, bias=True)
    (1): GELU(approximate='none')
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=672, out_features=672, bias=True)
    (4): GELU(approximate='none')
    (5): Dropout(p=0.1, inplace=False)
    (6): Linear(in_features=672, out_features=672, bias=True)
    (7): GELU(approximate='none')
    (8): Dropout(p=0.1, inplace=False)
    (9): Linear(in_features=672, out_features=672, bias=True)
  )
  (basal_encoder): Linear(in_features=18080, out_features=672, bias=True)
  (transformer_backbone): LlamaModel(
    (embed_tokens): Embedding(32000, 672, padding_idx=0)
    (layers): ModuleList(
      (0-3): 4 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
   

In [1]:
! uv run state tx predict --output-dir competition/first_run/ --checkpoint step=8000.ckpt

INFO:state._cli._tx._predict:Loaded config from competition/first_run/config.yaml
INFO:cell_load.data_modules.perturbation_dataloader:Loaded data module configuration from competition/first_run/data_module.torch
./competition_support_set/{competition_train,k562_gwps,rpe1,jurkat,k562,hepg2}.h5
INFO:cell_load.config:Configuration validation passed
INFO:cell_load.data_modules.perturbation_dataloader:Initializing DataModule: batch_size=16, workers=8, random_seed=42
INFO:cell_load.data_modules.perturbation_dataloader:Set 2 missing perturbations to zero vectors.
INFO:cell_load.data_modules.perturbation_dataloader:Loaded custom perturbation featurizations for 19792 perturbations.
INFO:cell_load.data_modules.perturbation_dataloader:Processing dataset replogle_h1:
INFO:cell_load.data_modules.perturbation_dataloader:  - Training dataset: True
INFO:cell_load.data_modules.perturbation_dataloader:  - Zeroshot cell types: ['hepg2']
INFO:cell_load.data_modules.perturbation_dataloader:  - Fewshot cell

In [2]:
# Run inference on validation anndata - make sure the checkpoint exists
! WANDB_MODE=offline uv run state tx infer \
  --output "competition/first_run/prediction_36000.h5ad" \
  --model-dir "competition/first_run" \
  --checkpoint "competition/first_run/checkpoints/step=36000.ckpt" \
  --adata "competition_support_set/competition_val_template.h5ad" \
  --pert-col "target_gene"

==> STATE: tx infer (virtual experiment)
Loaded config: competition/first_run/config.yaml
Control perturbation: non-targeting
Grouping by cell type column: cell_type
StateTransitionPerturbationModel(
  (loss_fn): SamplesLoss()
  (pert_encoder): Sequential(
    (0): Linear(in_features=5120, out_features=672, bias=True)
    (1): GELU(approximate='none')
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=672, out_features=672, bias=True)
    (4): GELU(approximate='none')
    (5): Dropout(p=0.1, inplace=False)
    (6): Linear(in_features=672, out_features=672, bias=True)
    (7): GELU(approximate='none')
    (8): Dropout(p=0.1, inplace=False)
    (9): Linear(in_features=672, out_features=672, bias=True)
  )
  (basal_encoder): Linear(in_features=18080, out_features=672, bias=True)
  (transformer_backbone): LlamaModel(
    (embed_tokens): Embedding(32000, 672, padding_idx=0)
    (layers): ModuleList(
      (0-3): 4 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
   

In [3]:
! uv run state tx predict --output-dir competition/first_run/ --checkpoint step=36000.ckpt

INFO:state._cli._tx._predict:Loaded config from competition/first_run/config.yaml
INFO:cell_load.data_modules.perturbation_dataloader:Loaded data module configuration from competition/first_run/data_module.torch
./competition_support_set/{competition_train,k562_gwps,rpe1,jurkat,k562,hepg2}.h5
INFO:cell_load.config:Configuration validation passed
INFO:cell_load.data_modules.perturbation_dataloader:Initializing DataModule: batch_size=16, workers=8, random_seed=42
INFO:cell_load.data_modules.perturbation_dataloader:Set 2 missing perturbations to zero vectors.
INFO:cell_load.data_modules.perturbation_dataloader:Loaded custom perturbation featurizations for 19792 perturbations.
INFO:cell_load.data_modules.perturbation_dataloader:Processing dataset replogle_h1:
INFO:cell_load.data_modules.perturbation_dataloader:  - Training dataset: True
INFO:cell_load.data_modules.perturbation_dataloader:  - Zeroshot cell types: ['hepg2']
INFO:cell_load.data_modules.perturbation_dataloader:  - Fewshot cell

In [6]:
# Run inference on validation anndata - make sure the checkpoint exists
! WANDB_MODE=offline uv run state tx infer \
  --output "competition/first_run/prediction_24000.h5ad" \
  --model-dir "competition/first_run" \
  --checkpoint "competition/first_run/checkpoints/step=24000.ckpt" \
  --adata "competition_support_set/competition_val_template.h5ad" \
  --pert-col "target_gene"

==> STATE: tx infer (virtual experiment)
Loaded config: competition/first_run/config.yaml
Control perturbation: non-targeting
Grouping by cell type column: cell_type
StateTransitionPerturbationModel(
  (loss_fn): SamplesLoss()
  (pert_encoder): Sequential(
    (0): Linear(in_features=5120, out_features=672, bias=True)
    (1): GELU(approximate='none')
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=672, out_features=672, bias=True)
    (4): GELU(approximate='none')
    (5): Dropout(p=0.1, inplace=False)
    (6): Linear(in_features=672, out_features=672, bias=True)
    (7): GELU(approximate='none')
    (8): Dropout(p=0.1, inplace=False)
    (9): Linear(in_features=672, out_features=672, bias=True)
  )
  (basal_encoder): Linear(in_features=18080, out_features=672, bias=True)
  (transformer_backbone): LlamaModel(
    (embed_tokens): Embedding(32000, 672, padding_idx=0)
    (layers): ModuleList(
      (0-3): 4 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
   

In [1]:
! uv run state tx predict --output-dir competition/first_run/ --checkpoint step=24000.ckpt

INFO:state._cli._tx._predict:Loaded config from competition/first_run/config.yaml
INFO:cell_load.data_modules.perturbation_dataloader:Loaded data module configuration from competition/first_run/data_module.torch
./competition_support_set/{competition_train,k562_gwps,rpe1,jurkat,k562,hepg2}.h5
INFO:cell_load.config:Configuration validation passed
INFO:cell_load.data_modules.perturbation_dataloader:Initializing DataModule: batch_size=16, workers=8, random_seed=42
INFO:cell_load.data_modules.perturbation_dataloader:Set 2 missing perturbations to zero vectors.
INFO:cell_load.data_modules.perturbation_dataloader:Loaded custom perturbation featurizations for 19792 perturbations.
INFO:cell_load.data_modules.perturbation_dataloader:Processing dataset replogle_h1:
INFO:cell_load.data_modules.perturbation_dataloader:  - Training dataset: True
INFO:cell_load.data_modules.perturbation_dataloader:  - Zeroshot cell types: ['hepg2']
INFO:cell_load.data_modules.perturbation_dataloader:  - Fewshot cell

In [2]:
# Run inference on validation anndata - make sure the checkpoint exists
! WANDB_MODE=offline uv run state tx infer \
  --output "competition/first_run/prediction_12000.h5ad" \
  --model-dir "competition/first_run" \
  --checkpoint "competition/first_run/checkpoints/step=12000.ckpt" \
  --adata "competition_support_set/competition_val_template.h5ad" \
  --pert-col "target_gene"

==> STATE: tx infer (virtual experiment)
Loaded config: competition/first_run/config.yaml
Control perturbation: non-targeting
Grouping by cell type column: cell_type
StateTransitionPerturbationModel(
  (loss_fn): SamplesLoss()
  (pert_encoder): Sequential(
    (0): Linear(in_features=5120, out_features=672, bias=True)
    (1): GELU(approximate='none')
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=672, out_features=672, bias=True)
    (4): GELU(approximate='none')
    (5): Dropout(p=0.1, inplace=False)
    (6): Linear(in_features=672, out_features=672, bias=True)
    (7): GELU(approximate='none')
    (8): Dropout(p=0.1, inplace=False)
    (9): Linear(in_features=672, out_features=672, bias=True)
  )
  (basal_encoder): Linear(in_features=18080, out_features=672, bias=True)
  (transformer_backbone): LlamaModel(
    (embed_tokens): Embedding(32000, 672, padding_idx=0)
    (layers): ModuleList(
      (0-3): 4 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
   

In [4]:
! uv run state tx predict --output-dir competition/first_run/ --checkpoint step=12000.ckpt

INFO:state._cli._tx._predict:Loaded config from competition/first_run/config.yaml
INFO:cell_load.data_modules.perturbation_dataloader:Loaded data module configuration from competition/first_run/data_module.torch
./competition_support_set/{competition_train,k562_gwps,rpe1,jurkat,k562,hepg2}.h5
INFO:cell_load.config:Configuration validation passed
INFO:cell_load.data_modules.perturbation_dataloader:Initializing DataModule: batch_size=16, workers=8, random_seed=42
INFO:cell_load.data_modules.perturbation_dataloader:Set 2 missing perturbations to zero vectors.
INFO:cell_load.data_modules.perturbation_dataloader:Loaded custom perturbation featurizations for 19792 perturbations.
INFO:cell_load.data_modules.perturbation_dataloader:Processing dataset replogle_h1:
INFO:cell_load.data_modules.perturbation_dataloader:  - Training dataset: True
INFO:cell_load.data_modules.perturbation_dataloader:  - Zeroshot cell types: ['hepg2']
INFO:cell_load.data_modules.perturbation_dataloader:  - Fewshot cell

In [5]:
! uv run state tx predict --output-dir competition/first_run/ --checkpoint step=8000.ckpt --eval-train-data

INFO:state._cli._tx._predict:Loaded config from competition/first_run/config.yaml
INFO:cell_load.data_modules.perturbation_dataloader:Loaded data module configuration from competition/first_run/data_module.torch
./competition_support_set/{competition_train,k562_gwps,rpe1,jurkat,k562,hepg2}.h5
INFO:cell_load.config:Configuration validation passed
INFO:cell_load.data_modules.perturbation_dataloader:Initializing DataModule: batch_size=16, workers=8, random_seed=42
INFO:cell_load.data_modules.perturbation_dataloader:Set 2 missing perturbations to zero vectors.
INFO:cell_load.data_modules.perturbation_dataloader:Loaded custom perturbation featurizations for 19792 perturbations.
INFO:cell_load.data_modules.perturbation_dataloader:Processing dataset replogle_h1:
INFO:cell_load.data_modules.perturbation_dataloader:  - Training dataset: True
INFO:cell_load.data_modules.perturbation_dataloader:  - Zeroshot cell types: ['hepg2']
INFO:cell_load.data_modules.perturbation_dataloader:  - Fewshot cell

In [1]:
!ls

all_score.py		   competition_hepg2_official	   ruff.toml
assets			   competition_support_set	   run3090.ipynb
baseline		   competition_support_set.zip	   scan1.py
baseline2		   configs			   scripts
baseline2_eval		   debug_eval.py		   server_train_2.py
baseline_eval		   debug_pred.h5ad		   server_train.py
batch_eval.py		   diagnose_overlap.py		   singularity.def
batch_process.py	   examples			   src
batch_results		   LICENSE			   tests
cell-eval-outdir	   MODEL_ACCEPTABLE_USE_POLICY.md  testtt.ipynb
check_competition_data.sh  MODEL_LICENSE.md		   train_and_eval.py
check_h5_files.py	   my_real_data.h5ad		   training.out
competition		   official-result		   Untitled.ipynb
competition_40000	   --outdir			   uv.lock
competition_40000_500	   pyproject.toml		   wandb
competition_hepg2	   README.md			   wandb_logs
competition_hepg2_2	   recover_missing_logs.py


In [2]:
! uv run state tx predict --output-dir competition/first_run/ --checkpoint step=12000.ckpt --eval-train-data

INFO:state._cli._tx._predict:Loaded config from competition/first_run/config.yaml
INFO:cell_load.data_modules.perturbation_dataloader:Loaded data module configuration from competition/first_run/data_module.torch
./competition_support_set/{competition_train,k562_gwps,rpe1,jurkat,k562,hepg2}.h5
INFO:cell_load.config:Configuration validation passed
INFO:cell_load.data_modules.perturbation_dataloader:Initializing DataModule: batch_size=16, workers=8, random_seed=42
INFO:cell_load.data_modules.perturbation_dataloader:Set 2 missing perturbations to zero vectors.
INFO:cell_load.data_modules.perturbation_dataloader:Loaded custom perturbation featurizations for 19792 perturbations.
INFO:cell_load.data_modules.perturbation_dataloader:Processing dataset replogle_h1:
INFO:cell_load.data_modules.perturbation_dataloader:  - Training dataset: True
INFO:cell_load.data_modules.perturbation_dataloader:  - Zeroshot cell types: ['hepg2']
INFO:cell_load.data_modules.perturbation_dataloader:  - Fewshot cell

In [ ]:
! uv run state tx predict --output-dir competition/first_run/ --checkpoint step=20000.ckpt --eval-train-data

INFO:state._cli._tx._predict:Loaded config from competition/first_run/config.yaml
INFO:cell_load.data_modules.perturbation_dataloader:Loaded data module configuration from competition/first_run/data_module.torch
./competition_support_set/{competition_train,k562_gwps,rpe1,jurkat,k562,hepg2}.h5
INFO:cell_load.config:Configuration validation passed
INFO:cell_load.data_modules.perturbation_dataloader:Initializing DataModule: batch_size=16, workers=8, random_seed=42
INFO:cell_load.data_modules.perturbation_dataloader:Set 2 missing perturbations to zero vectors.
INFO:cell_load.data_modules.perturbation_dataloader:Loaded custom perturbation featurizations for 19792 perturbations.
INFO:cell_load.data_modules.perturbation_dataloader:Processing dataset replogle_h1:
INFO:cell_load.data_modules.perturbation_dataloader:  - Training dataset: True
INFO:cell_load.data_modules.perturbation_dataloader:  - Zeroshot cell types: ['hepg2']
INFO:cell_load.data_modules.perturbation_dataloader:  - Fewshot cell